In [1]:
from pymilvus import MilvusClient
client = MilvusClient("milvus_demo.db")
client.create_collection(
  collection_name="demo_collection",
  dimension=5
)

In [2]:
client

In [3]:
res = client.insert(
  collection_name="demo_collection",
  data=[
    {"id": 0, "vector": [0.1, 0.2, 0.3, 0.2, 0.1], "text": "AI was proposed in 1956.", "subject": "history"},
    {"id": 1, "vector": [0.1, 0.2, 0.3, 0.2, 0.1], "text": "Alan Turing was born in London.", "subject": "history"},
    {"id": 2, "vector": [0.1, 0.2, 0.3, 0.2, 0.1], "text": "Computational synthesis with AI algorithms predicts molecular properties.", "subject": "biology"},
  ]
)

In [7]:
res = client.search(
  collection_name="demo_collection",
  data=[[0.1, 0.2, 0.3, 0.2, 0.1]],
  filter="subject == 'history'",
  limit=2,
  output_fields=["text", "subject"],
)

In [8]:
res

data: [[{'id': 1, 'distance': 1.0, 'entity': {'text': 'Alan Turing was born in London.', 'subject': 'history'}}, {'id': 0, 'distance': 1.0, 'entity': {'text': 'AI was proposed in 1956.', 'subject': 'history'}}]]

In [ ]:
from pymilvus import MilvusClient
client = MilvusClient("milvus_demo.db")
client.create_collection(
  collection_name="demo_collection",
  dimension=5
)

In [1]:
import os
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings # Or your preferred embeddings
from langchain_community.vectorstores import Milvus

# --- 1. Configuration ---

# Define a local file to store Milvus Lite data (optional)
# If omitted, pymilvus[lite] often uses a default file like './milvus_demo.db'
# or a temporary location depending on the exact version and setup.
# Specifying a file makes persistence clear.
LOCAL_MILVUS_FILE = "my_milvus_lite_data.db"

COLLECTION_NAME = "LangChainLiteDemo"

# Set up your embedding model
# Ensure OPENAI_API_KEY is set as an environment variable if using OpenAI
embeddings = OpenAIEmbeddings()

# --- 2. Sample Data ---
docs = [
    Document(page_content="Milvus Lite runs embedded in your Python application."),
    Document(page_content="It stores data locally, without needing a separate server."),
    Document(page_content="LangChain integrates smoothly with Milvus Lite."),
    Document(page_content="This is great for quick prototypes and local development."),
]

# --- 3. Initialize Milvus Vector Store using Milvus Lite ---
# Notice NO 'connection_args' for host/port are needed.
# We pass the local file path using 'uri' argument within connection_args
# Or often, just omitting connection_args works if relying on defaults.
# For clarity and explicit persistence, using 'uri' is recommended.

print(f"Initializing Milvus Lite store. Data will be stored in: {LOCAL_MILVUS_FILE}")
print(f"Using collection: '{COLLECTION_NAME}'")

try:
    vector_store = Milvus.from_documents(
        documents=docs,
        embedding=embeddings,
        collection_name=COLLECTION_NAME,
        # Key part for Milvus Lite: Specify the local file via 'uri'
        connection_args={"uri": LOCAL_MILVUS_FILE},
        # drop_old=True # Uncomment to clear data in the file on restart (USE WITH CAUTION!)
    )
    print("Successfully initialized Milvus Lite vector store and added documents.")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Ensure 'pymilvus[lite]' is installed correctly.")
    exit()

# --- 4. Perform Similarity Search ---
query = "How does Milvus Lite store data?"
print(f"\nPerforming similarity search for: '{query}'")

try:
    search_results = vector_store.similarity_search(query, k=2)

    print("\nSearch Results:")
    if search_results:
        for i, doc in enumerate(search_results):
            print(f"{i+1}. Content: {doc.page_content}")
    else:
        print("No results found.")

except Exception as e:
    print(f"An error occurred during search: {e}")

print("\nExample finished.")
# The data persists in the 'my_milvus_lite_data.db' file (or the default location)
# Running the script again (without drop_old=True) will load the existing data.

Initializing Milvus Lite store. Data will be stored in: my_milvus_lite_data.db
Using collection: 'LangChainLiteDemo'


Invalid Milvus URI: my_milvus_lite_data.db


An error occurred: ('Invalid Milvus URI: %s', 'my_milvus_lite_data.db')
Ensure 'pymilvus[lite]' is installed correctly.

Performing similarity search for: 'How does Milvus Lite store data?'
An error occurred during search: name 'vector_store' is not defined

Example finished.


: 